In [1]:
import os

In [2]:
directory = "results/classifications/20210802/"
suffix = '20210802'

### Rename files

In [ ]:
# for file in os.listdir(directory):
#     #print(file[0:58]+'20201215.tif')
#     os.rename(directory+file, directory+file[0:58]+'20201215.tif')

## Mosiac tifs

In [ ]:
#build vrt
os.chdir(directory)
os.system("gdalbuildvrt " + "Northern_gm_mads_two_seasons_"+suffix+"_mosaic.vrt *.tif")

#build mosaic from .vrt
os.system("gdal_translate "\
       "-co BIGTIFF=YES "\
       "-co COMPRESS=DEFLATE "\
       "-co ZLEVEL=9 "\
       "-co PREDICTOR=1 "\
       "-co TILED=YES "\
       "-co BLOCKXSIZE=1024 "\
       "-co BLOCKYSIZE=1024 "\
       "-co NUM_THREADS=25 "\
       "--config GDAL_CACHEMAX 512 "
       + "Northern_gm_mads_two_seasons_"+suffix+"_mosaic.vrt " + "Northern_gm_mads_two_seasons_"+suffix+"_mosaic.tif")

## Clip mosaic to AEZ

In [3]:
import xarray as xr
import numpy as np
import geopandas as gpd
from datacube.utils.cog import write_cog
import sys

from deafrica_tools.spatial import xr_rasterize
from deafrica_tools.dask import create_local_dask_cluster

/env/lib/python3.6/site-packages/geopandas/_compat.py:88: UserWarning: The Shapely GEOS version (3.7.2-CAPI-1.11.0 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.1-CAPI-1.14.2). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


In [4]:
create_local_dask_cluster()

Client Scheduler: tcp://127.0.0.1:36651 Dashboard: /user/chad/proxy/8787/status,Cluster Workers: 1 Cores: 31 Memory: 254.70 GB


In [5]:
aez_region = 'data/Northern.geojson'
pred_tif = "results/classifications/"+suffix+"/Northern_gm_mads_two_seasons_"+suffix+"_mosaic.tif"
chunks={'x':5000, 'y':5000}

In [6]:
aez = gpd.read_file(aez_region)
da = xr.open_rasterio(pred_tif)

In [7]:
aez = xr_rasterize(aez, da)

/env/lib/python3.6/site-packages/pyproj/crs/crs.py:280: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  projstring = _prepare_from_string(projparams)


In [8]:
da = da.chunk(chunks).where(aez.chunk(chunks)).astype(np.int8)

In [9]:
%%time
da = da.compute()

CPU times: user 35.7 s, sys: 1min 11s, total: 1min 47s
Wall time: 3min 12s


In [10]:
write_cog(da.squeeze(),
          pred_tif[:-4]+'_clipped.tif',
          overwrite=True)

PosixPath('results/classifications/20210802/Northern_gm_mads_two_seasons_20210802_mosaic_clipped.tif')

In [ ]:
# write_cog(aez.squeeze(),
#           'data/WDPA_eastern.tif',
#           overwrite=True)